In [1]:
# This journal is a stress-testing of the models trained using siku. 

# Typical data processing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import root_mean_squared_error, r2_score


# Additional helper libraries
from joblib import load, dump # For saving the model after training

In [46]:
# First we need to load our trained models using joblib
fullModel = load('../sikuOutput/Canada_full_dataset_model.joblib')
below100kModel = load('../sikuOutput/Canada_below_100k_model.joblib')

# We'll use the provincial datasets to test the models. Let's load them all into a list.
years = [2016, 2021]
locationList = ['PEI', 'NS', 'NB', 'NL', 'QC', 'ON', 'MB', 'SK', 'AB', 'BC', 'YK', 'NT', 'NU']
import2016 = []
import2021 = []

for location in locationList:
    for year in years:
        locationData = pd.read_csv(f'../processedData/processed_{location}_{year}.csv')
        entry = { 'name': location, 'data': locationData}
        if year == 2016:
            import2016.append(entry)
        else:
            import2021.append(entry)


c:\Users\danes\Documents\ACENETMicrocredentialinAdvancedComputing\ISP\CommunityGrowthPrediction\ispEnv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.5.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\danes\Documents\ACENETMicrocredentialinAdvancedComputing\ISP\CommunityGrowthPrediction\ispEnv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\danes\Documents\ACENETMicrocredentialinAdvan

In [60]:
# We can use our previous method to prepare each file for prediction

def dataPrep(inputData, targetData = import2021[0]['data']):
    '''
    This function takes in two dataframes, one for the input data and one for population targets, and prepares them for prediction using the models.
    Args:
    inputData: A pandas dataframe containing the 2016 data for a province
    targetData: A pandas dataframe containing the 2021 data for a province
    '''

    # The 2021 data only needs the GEO_NAME for cross referencing, the population count for training,
    # and the province for handling duplicate community names
    trimmedTargets = targetData[['GEO_NAME', 'Population, 2021']]
    # Drop any rows with missing population data
    trimmedTargets = trimmedTargets.dropna()
    # Sort the data by community name
    trimmedTargets = trimmedTargets.sort_values(by='GEO_NAME')
    # Take a subset of the 2021 data that matches 
    # the communities in the 2016 data. Note that community name is 'GEO_NAME' in both datasets.
    trimmedTargets = trimmedTargets.merge(inputData[['GEO_NAME']], on=['GEO_NAME'], how='inner')

    # Now let's trim the 2016 data to only include what has a match in our trimmed 2021 data
    trimmedSource = inputData.merge(trimmedTargets[['GEO_NAME']], on=['GEO_NAME'], how='inner')

    # As a last step before we are done, we need to sort the dataframes by the community name so that the
    # data is in the same order for both datasets
    trimmedSource = trimmedSource.sort_values(by='GEO_NAME')
    trimmedTargets = trimmedTargets.sort_values(by='GEO_NAME')
    return trimmedSource, trimmedTargets


In [61]:
def predictProvinces(sourceData, targetData, model = fullModel):
    '''
    This predicts population change for all the provinces in a given source dataset, and returns the results in a list.

    Args:
    sourceData: A list of dictionaries containing the name of the province and the data for that province
    targetData: A list of dictionaries containing the name of the province and the data for that province, for the prediction year
    model: The model to use for prediction. Default is the full model trained on the entire dataset.

    Returns:
    trimmedData: A list of dictionaries containing the name of the province, the source data, the target data, and the predicted population change.
    '''
    results = []
    for i in range(len(sourceData)):
        trimmedSource, trimmedTarget = dataPrep(sourceData[i]['data'], targetData[i]['data'])
        predicted = model.predict(trimmedSource.drop(columns=['GEO_NAME']))
        r2 = r2_score(trimmedTarget['Population, 2021'], predicted)
        rmse = root_mean_squared_error(trimmedTarget['Population, 2021'], predicted)
        results.append(
            {
                'name': sourceData[i]['name'], 
                'sourceData': trimmedSource, 
                'targetData': trimmedTarget,
                'predicted': predicted,
                'r2': r2,
                'rmse': rmse
            }
        )
    return results

In [63]:
# Predict the population changes for all provinces
results = predictProvinces(import2016, import2021)

In [64]:
for result in results:
    print(f"{result['name']}: R2 = {result['r2']}, RMSE = {result['rmse']}, 'Percent Error' = {result['rmse']/result['targetData']['Population, 2021'].mean()}")
    print("\n\n\n")

PEI: R2 = 0.9789047504133078, RMSE = 955.2565263726285, 'Percent Error' = 0.41183726077716254




NS: R2 = 0.9994874038993332, RMSE = 1311.7080894138799, 'Percent Error' = 0.09808035672654741




NB: R2 = 0.12445167787983846, RMSE = 6586.067475539386, 'Percent Error' = 2.4504331799584005




NL: R2 = 0.9987204278347929, RMSE = 259.81189738768245, 'Percent Error' = 0.15586643126773347




QC: R2 = -3627.0503369858634, RMSE = 107395.97253974681, 'Percent Error' = 30.58842852171655




ON: R2 = 0.7181467311737502, RMSE = 75170.74080829739, 'Percent Error' = 2.889824134188699




MB: R2 = 0.9997622125732802, RMSE = 829.9431741698203, 'Percent Error' = 0.12584262162040466




SK: R2 = 0.9969995177700325, RMSE = 638.3166567215249, 'Percent Error' = 0.5291770473928105




AB: R2 = 0.9943200228888494, RMSE = 6255.614849681326, 'Percent Error' = 0.592336585468474




BC: R2 = 0.9935914577713143, RMSE = 3116.4022308127546, 'Percent Error' = 0.43687550448480456




YK: R2 = 0.9997923455771118, RM

In [65]:
# Now let's trim down the dataset to only include communities with a population of less than 100,000
# and redo our prediction using the below100k model
# If we trim out the 2016 data, the dataPrep function will do the work for the 2021 data

below100k2016 = []

for entry in import2016:
    below100k2016.append({'name': entry['name'], 'data': entry['data'][entry['data']['Population, 2016'] < 100000]})
    

resultsBelow100k = predictProvinces(below100k2016, import2021, model = below100kModel)





In [66]:
print("Below 100k")
for result in resultsBelow100k:
    print(f"{result['name']}: R2 = {result['r2']}, RMSE = {result['rmse']}, 'Percent Error' = {result['rmse']/result['targetData']['Population, 2021'].mean()}")
    print("\n\n\n")

Below 100k
PEI: R2 = 0.9131748469208097, RMSE = 1937.9852097905605, 'Percent Error' = 0.8355185211427293




NS: R2 = 0.9425299950789617, RMSE = 3117.0646276747902, 'Percent Error' = 0.5290050192563107




NB: R2 = 0.9894997571362266, RMSE = 721.2499363700873, 'Percent Error' = 0.26835054175927686




NL: R2 = 0.9783664578824616, RMSE = 461.71590544423333, 'Percent Error' = 0.36284880796180097




QC: R2 = -10.879819390034767, RMSE = 6145.486404252005, 'Percent Error' = 1.7503521515955582




ON: R2 = 0.9625520779713657, RMSE = 3062.6163918508896, 'Percent Error' = 0.34106651858516707




MB: R2 = 0.9800608440939055, RMSE = 654.6344156053409, 'Percent Error' = 0.24021215261727386




SK: R2 = 0.9530910952725716, RMSE = 453.67723549239844, 'Percent Error' = 0.6736200342359518




AB: R2 = 0.9650872299131489, RMSE = 2147.976159310979, 'Percent Error' = 0.47221651955848676




BC: R2 = 0.9422236371021022, RMSE = 2707.043296295594, 'Percent Error' = 0.7653022652710367




YK: R2 = 0.969487